In [ ]:
#%%

# Step 1: Read data
import pandas as pd
transfusion = pd.read_csv("transfusion.data")
print(transfusion.head())

   Recency (months)  Frequency (times)  Monetary (c.c. blood)  Time (months)  \
0                 2                 50                  12500             98   
1                 0                 13                   3250             28   
2                 1                 16                   4000             35   
3                 2                 20                   5000             45   
4                 1                 24                   6000             77   

   whether he/she donated blood in March 2007  
0                                           1  
1                                           1  
2                                           1  
3                                           1  
4                                           0  


In [ ]:
#%%

# Step 2: Data info
print(transfusion.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB
None


In [ ]:
#%%

# Step 3: Rename target column
transfusion.rename(columns={'whether he/she donated blood in March 2007': 'target'}, inplace=True)
print(transfusion.head(2))

   Recency (months)  Frequency (times)  Monetary (c.c. blood)  Time (months)  \
0                 2                 50                  12500             98   
1                 0                 13                   3250             28   

   target  
0       1  
1       1  


In [ ]:
#%%

# Step 4: Target distribution
print(transfusion['target'].value_counts(normalize=True).round(3))

target
0    0.762
1    0.238
Name: proportion, dtype: float64


In [ ]:
#%%

# Step 5: Initial split
from sklearn.model_selection import train_test_split

X = transfusion.drop('target', axis=1)
y = transfusion['target']

# Split into temp and final test set
X_temp, X_reserva, y_temp, y_reserva = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

In [ ]:
#%%

# Step 6: Train-validation split
X_train, X_test, y_train, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.25,
    random_state=42,
    stratify=y_temp
)

No pude hacer correr la librería de tpot, lo intenté lo más que pude pero nada funcionó, lo siento

In [ ]:
#%%

# Step 7: TPOT setup 
#import os
#os.environ["TPOT_NO_DASK"] = "true"
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score

# Updated TPOT configuration
tpot = TPOTClassifier(
    generations=5,
    population_size=20,
    cv=5,
    scorers=['roc_auc'],  # Correct parameter name for scoring
    scorers_weights=[1.0],  # Weight for the scorer
    verbose=2,
    random_state=42,
    n_jobs=-1
)

tpot.fit(X_train, y_train)

# Evaluation
tpot_auc = roc_auc_score(y_test, tpot.predict_proba(X_test)[:, 1])
print(f"TPOT AUC: {tpot_auc:.4f}")

/opt/homebrew/Caskroom/miniconda/base/envs/tf/lib/python3.10/site-packages/tpot/tpot_estimator/estimator.py:456: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")


TimeoutError: No valid workers found

In [ ]:
#%%

# Step 8: Feature engineering
import numpy as np

# Log-transform high-variance feature
col_to_normalize = X_train.var().idxmax()
for df in [X_train, X_test, X_temp, X_reserva]:
    df['monetary_log'] = np.log(df[col_to_normalize])
    df.drop(col_to_normalize, axis=1, inplace=True)

: 

In [ ]:
#%%

# Step 9: Logistic regression baseline
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='saga', max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)
logreg_auc = roc_auc_score(y_test, logreg.predict_proba(X_test)[:, 1])
print(f"Baseline AUC: {logreg_auc:.4f}")

: 

In [ ]:
#%%

# Step 10: Hyperparameter tuning (fixed data usage)
from sklearn.model_selection import GridSearchCV

# Updated parameter grid with valid combinations
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['saga'],  # Only solver supporting all penalties
    'l1_ratio': [0.3, 0.5, 0.7]
}

logreg_cv = GridSearchCV(
    LogisticRegression(max_iter=1000, random_state=42),
    param_grid,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1
)

# Corrected to use training data
logreg_cv.fit(X_train, y_train)

print(f"Best params: {logreg_cv.best_params_}")
best_auc = roc_auc_score(y_test, logreg_cv.predict_proba(X_test)[:, 1])
print(f"Tuned AUC: {best_auc:.4f}")

: 

In [ ]:
#%%

# Step 11: Final evaluation on holdout set
final_model = logreg_cv.best_estimator_
final_model.fit(X_temp, y_temp)  # Train on full temp data
reserva_auc = roc_auc_score(y_reserva, final_model.predict_proba(X_reserva)[:, 1])
print(f"Final Holdout AUC: {reserva_auc:.4f}")

: 